<a href="https://colab.research.google.com/github/madhumitha-r26/DL_Learnings/blob/main/keras_wide_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [ ]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tfB
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers= keras.layers

In [ ]:
print("TF VERSION:",tf.__version__)

TF VERSION: 2.19.0


In [ ]:
# Define the filename within the dataset
URL = "winemag-data-130k-v2.csv"

# Load the dataset by specifying the adapter, the dataset handle, and the filename within the dataset.
data = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS, # converting the kaggle dataset to pandas dataframe
  "zynicide/wine-reviews",
  URL)

/tmp/ipython-input-3933256722.py:5: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  data = kagglehub.load_dataset(


Using Colab cache for faster access to the 'wine-reviews' dataset.


In [ ]:
print("First 5 records:", data.head())

First 5 records:    Unnamed: 0   country                                        description  \
0           0     Italy  Aromas include tropical fruit, broom, brimston...   
1           1  Portugal  This is ripe and fruity, a wine that is smooth...   
2           2        US  Tart and snappy, the flavors of lime flesh and...   
3           3        US  Pineapple rind, lemon pith and orange blossom ...   
4           4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  

In [ ]:
data=data.sample(frac=1)  #shuffle data

In [ ]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
15194,15194,US,"Fairly light in color and body, this wine sing...",Rochioli-Allen Vineyards,95,70.0,California,Russian River Valley,Sonoma,Virginie Boone,@vboone,Gary Farrell 2013 Rochioli-Allen Vineyards Pin...,Pinot Noir,Gary Farrell
91426,91426,Greece,"Agiorgitiko, famed in Nemea and popular for it...",Réserve,87,16.0,Nemea,NaN,NaN,Susan Kostrzewa,@suskostrzewa,Tsantali 2003 Réserve Agiorgitiko (Nemea),Agiorgitiko,Tsantali
46929,46929,Spain,"A crisp combination of mineral notes, citrus a...",Ilercavònia,88,17.0,Catalonia,Terra Alta,NaN,Michael Schachner,@wineschach,Altavins 2015 Ilercavònia Grenache Blanc (Terr...,Grenache Blanc,Altavins
104841,104841,US,"Crisp, elegant black-plum and cassis flavors a...",Tuthills Lane Vineyard,89,60.0,New York,North Fork of Long Island,Long Island,Anna Lee C. Iijima,NaN,Paumanok 2005 Tuthills Lane Vineyard Cabernet ...,Cabernet Sauvignon,Paumanok
92082,92082,US,"Picture perfect Sonoma Zin, said to be made fr...",Forgotten Vines,92,34.0,California,Sonoma County,Sonoma,NaN,NaN,De Loach 2009 Forgotten Vines Zinfandel (Sonom...,Zinfandel,De Loach


data preprocessing to limit the number of wine varities in the dataset

In [ ]:
data=data[pd.notnull(data['country'])]
data=data[pd.notnull(data['price'])]
data=data.drop(data.columns[0],axis=1)

In [ ]:
variety_threshold=500  #anything that occurs less than this will get removed
value_counts=data['variety'].value_counts()
to_remove=value_counts[value_counts<=variety_threshold].index
data.replace(to_remove,np.nan,inplace=True)
data=data[pd.notnull(data['variety'])]

split data into train and test

In [ ]:
len(data)   #total number of records in the dataset

103039

In [ ]:
train_size=int(len(data)*0.8)
print("train size: %d" % train_size)
print("test size: %d" % (len(data)-train_size))

train size: 82431
test size: 20608


In [ ]:
#train features
description_train=data['description'][:train_size]
variety_train=data['variety'][:train_size]

#train labels
label_train=data['price'][:train_size]

#test features
description_test=data['description'][train_size:]
variety_test=data['variety'][train_size:]

#test labels
label_test=data['price'][train_size:]

create a tokenizer to preprocess the text descriptions

In [ ]:
vocab_size=12000                #hyperparameter that used to experiment with different values for our dataset
tokenize=keras.preprocessing.text.Tokenizer(num_words=vocab_size,char_level=False)
tokenize.fit_on_texts(description_train)            #only fit on the train

**define wide features**

**wide feature-1:**
    sparse bag of words (bow) vocab_size vector

In [ ]:
tokenize=keras.preprocessing.text.Tokenizer(num_words=vocab_size,char_level=False)
tokenize.fit_on_texts(description_train)

descriptor_bow_train=tokenize.texts_to_matrix(description_train)
descriptor_bow_test=tokenize.texts_to_matrix(description_test)

**wide feature-2:**
    one hot vector of variety categories

sklearn utility - convert label strings to numbered index

In [ ]:
encoder=LabelEncoder()
encoder.fit(variety_train)
variety_train=encoder.transform(variety_train)
variety_test=encoder.transform(variety_test)
num_classes=np.max(variety_train)+1

#convert labels to one hot
variety_train=keras.utils.to_categorical(variety_train,num_classes)
variety_test=keras.utils.to_categorical(variety_test,num_classes)

**define wide model with functional api**

functional api is flexible to use



In [ ]:
bow_inputs=layers.Input(shape=(vocab_size,))
variety_inputs=layers.Input(shape=(num_classes,))
merged_layer=layers.concatenate([bow_inputs,variety_inputs])
merged_layer=layers.Dense(256,activation='relu')(merged_layer)
predictions=layers.Dense(1)(merged_layer)
wide_model=keras.Model(inputs=[bow_inputs,variety_inputs],outputs=predictions)

In [ ]:
wide_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
print(wide_model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 38)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 12038)     │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  3,081,984 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        257 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,082,241 (11.76 MB)

 Trainable params: 3,082,241 (11.76 MB)

 Non-trainable params: 0 (0.00 B)

None


**define deep model with functional api**

**deep model feature:**
    word embeddings of wine descriptions

In [ ]:
trained_embed=tokenize.texts_to_sequences(description_train)
test_embed=tokenize.texts_to_sequences(description_test)

max_seq_length=170
trained_embed=keras.preprocessing.sequence.pad_sequences(trained_embed,maxlen=max_seq_length,padding="post")
test_embed=keras.preprocessing.sequence.pad_sequences(test_embed,maxlen=max_seq_length,padding="post")

In [ ]:
deep_inputs=layers.Input(shape=(max_seq_length,))
embedding=layers.Embedding(vocab_size,8,input_length=max_seq_length)(deep_inputs)
embedding=layers.Flatten()(embedding)
embed_out=layers.Dense(1)(embedding)
deep_model=keras.Model(inputs=deep_inputs,outputs=embed_out)
print(deep_model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 170)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 170, 8)         │        96,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1360)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,361 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,361 (380.32 KB)

 Trainable params: 97,361 (380.32 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
deep_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

combine wide and deep into one model


In [ ]:
merged_out=layers.concatenate([wide_model.output,deep_model.output])
merged_out=layers.Dense(1)(merged_out)
combined_model=keras.Model(wide_model.input+[deep_model.input],merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 38)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 170)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 12038)     │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 170, 8)    │     96,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  3,081,984 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1360)      │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        257 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │      1,361 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 2)         │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │          3 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,179,605 (12.13 MB)

 Trainable params: 3,179,605 (12.13 MB)

 Non-trainable params: 0 (0.00 B)

None


run training

In [ ]:
combined_model.fit([descriptor_bow_train,variety_train]+[trained_embed],label_train, epochs=10, batch_size=128)

In [ ]:
combined_model.fit([descriptor_bow_test,variety_test]+[test_embed],label_test, epochs=10, batch_size=128)

In [ ]:
predictions=combined_model.predict([description_test,variety_test]+[test_embed])

In [ ]:
num_predictions=40
diff=0

for i in range(num_predictions):
  val=predictions[i]
  print(description_test.iloc[i])
  print('Predicted: ',val[0],'Actual: ',label_test.iloc[i],'\n')
  diff+=abs(val[0]-label_test.iloc[i])